# 0- Imports

In [1]:
# Install dependencies
#!pip install -q amplpy

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Google Colab & Kaggle integration
from amplpy import AMPL, tools
ampl = tools.ampl_notebook(
    modules=["coin", "highs", "gokestrel", "gurobi", "cplex"], # modules to install
    license_uuid=licenca, # license to use
    g=globals()) # instantiate AMPL object and register magics

Licensed to AMPL Community Edition License for <candace.vasconcelos.101404@ga.ita.br>.


In [5]:
ampl.option["solver"] = "gurobi"

# 1- Modelagem

## 1.1- Bin-packing adaptado para Cutting Stock Problem (CSP)

Modelagem matemática 

Dado que temos $n$ tipos de itens para empacotar, cada um com peso $w_{i}$ e uma demanda inteira $d_{i}$, para todo $i \in \{1,...,n\}$, e um número $m$ de mochilas (caixas) idênticas com capacidade inteira $c$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} y_{j} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i} \xi_{i j} \leq c y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{j = 1}^{m} \xi_{i j} = d_{i} \quad \forall i \in \{1,...,n\} \\
&\xi_{i j} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} \\
&y_{j} \in \{0,1\} \quad \forall j \in \{1,...,m\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j}$= número de itens tipo $i$ adicionados na mochila $j$


In [6]:
%%ampl_eval
reset;

In [7]:
%%ampl_eval
param n;
param C;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
param w {I} > 0;
param d {I} > 0;
param maxVal := max {i in I} w[i];
param soma := sum{i in I} d[i];
param maxbins := ceil(soma / floor(C / maxVal));

set J = 1..maxbins;

var e {I,J} integer >= 0;
var y {J} binary;

minimize FO:  sum {j in J} y[j];

s.t. R1_capacidade_mochila {j in J}:
   sum {i in I} w[i] * e[i,j] <= C * y[j] suffix block j;

s.t. R2_quantidade_demanda {i in I}:
   sum {j in J} e[i, j] = d[i] suffix block i;


### 1.1.1- Importar dados

In [8]:
df_process = pd.read_excel('../01-Data/ordens_pre_process.xlsx')

df_process

,Id,Ordem de produção,Data da ordem de produção,Produto,Comprimento unit,Largura unit,Altura unit,Peso unit,Peças,Caixa padrão,...,Item pai,Cor,Tamanho,Inner,Rota,Capacidade,pai-cor,Volume unit,Volume total,Peso_total
0,0,2,2023-11-14,828646451,284,131,50,0.250,3,CARTON_M,...,2,2.0,2,1,11,2.720000e+10,2-2.0,1860200,5580600,0.750
1,1,3,2023-11-09,605410977,250,100,14,0.120,1,CARTON_M,...,3,3.0,3,1,5,2.720000e+10,3-3.0,350000,350000,0.120
2,2,7,2023-11-11,545612218,81,87,162,0.640,1,CARTON_S,...,7,NaN,7,1,1,inf,NaN,1141614,1141614,0.640
3,3,8,2023-11-24,861652334,248,216,128,0.300,5,CARTON_M,...,8,7.0,5,1,3,inf,8-7.0,6856704,34283520,1.500
4,4,9,2023-11-13,879695703,393,295,32,0.520,2,CARTON_M,...,9,8.0,8,1,11,2.720000e+10,9-8.0,3709920,7419840,1.040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58453,58453,68,2023-11-07,843672948,485,377,30,0.233,3,CARTON_M,...,353,4.0,1,1,1,inf,353-4.0,5485350,16456050,0.699
58454,58454,8926,2023-11-23,796269830,443,268,19,0.140,4,CARTON_M,...,958,2.0,14,1,4,inf,958-2.0,2255756,9023024,0.560
58455,58455,257,2023-11-07,611398476,412,281,27,0.440,1,CARTON_M,...,554,22.0,1,1,3,inf,554-22.0,3125844,3125844,0.440
58456,58456,8936,2023-11-24,882144507,387,283,18,0.107,4,CARTON_M,...,130,9.0,115,1,3,inf,130-9.0,1971378,7885512,0.428


In [9]:
df_process.columns = df_process.columns.str.replace(' ', '_')

In [10]:
df_process[['Loja','Tipo_de_buffer','Classe_de_onda']] = df_process[['Loja','Tipo_de_buffer', 'Classe_de_onda']].astype('string')

In [11]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58458 entries, 0 to 58457
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         58458 non-null  int64         
 1   Ordem_de_produção          58458 non-null  int64         
 2   Data_da_ordem_de_produção  58458 non-null  datetime64[ns]
 3   Produto                    58458 non-null  int64         
 4   Comprimento_unit           58458 non-null  int64         
 5   Largura_unit               58458 non-null  int64         
 6   Altura_unit                58458 non-null  int64         
 7   Peso_unit                  58458 non-null  float64       
 8   Peças                      58458 non-null  int64         
 9   Caixa_padrão               58458 non-null  object        
 10  Loja                       58458 non-null  string        
 11  Classe_de_onda             58458 non-null  string        
 12  Tipo

verificar a demanda por loja

In [12]:
df_process.groupby(['Tipo_de_buffer', 'Loja', 'Classe_de_onda']).agg({'Peças':'sum'}).head(30)

Peças
Tipo_de_buffer Loja Classe_de_onda       
CX             1    3                  11
                    4                  39
                    6                   5
                    9                   9
               10   1                1356
                    2                 499
                    3                 303
                    4                1081
                    5                 336
                    6                 421
                    7                 336
                    8                  12
                    9                 849
               100  2                   7
                    3                  27
                    4                  84
                    5                  12
                    6                  34
                    7                   4
                    9                  69
               101  2                  21
                    3                  33
                    4                 128
                    5                  36
                    6                  33
                    7                   2
                    9                  57
               102  3                  18
                    4                  46
                    5                  11

Vamos reduzir o problema para uma loja

In [13]:
df_process_situacao1 = df_process.query('Tipo_de_buffer == "CX" & Loja == "2" & Classe_de_onda == "3"')

with pd.option_context('display.max_columns', None):
    display(df_process_situacao1)


,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,Loja,Classe_de_onda,Tipo_de_buffer,Item_pai,Cor,Tamanho,Inner,Rota,Capacidade,pai-cor,Volume_unit,Volume_total,Peso_total
704,704,1237,2023-11-16,736849445,447,268,20,0.284,2,CARTON_M,2,3,CX,992,347.0,6,1,11,2.720000e+10,992-347.0,2395920,4791840,0.568
1439,1439,59,2023-11-14,841264182,437,192,16,0.218,5,CARTON_M,2,3,CX,59,37.0,3,1,11,2.720000e+10,59-37.0,1342464,6712320,1.090
1753,1753,987,2023-11-14,882057278,449,239,62,0.450,5,CARTON_M,2,3,CX,827,9.0,5,1,11,2.720000e+10,827-9.0,6653282,33266410,2.250
2191,2191,933,2023-11-14,856853019,564,277,24,0.442,2,CARTON_M,2,3,CX,792,4.0,6,1,11,2.720000e+10,792-4.0,3749472,7498944,0.884
2210,2210,989,2023-11-14,675645211,407,226,27,0.315,2,CARTON_M,2,3,CX,829,4.0,1,1,11,2.720000e+10,829-4.0,2483514,4967028,0.630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29881,29881,7585,2023-11-14,841264123,446,201,17,0.219,9,CARTON_M,2,3,CX,59,4.0,3,1,11,2.720000e+10,59-4.0,1523982,13715838,1.971
30129,30129,1723,2023-11-07,595813106,335,172,38,0.150,1,CARTON_M,2,3,CX,503,77.0,3,1,11,2.720000e+10,503-77.0,2189560,2189560,0.150
30672,30672,1534,2023-11-07,817850042,364,288,23,0.213,1,CARTON_M,2,3,CX,1155,137.0,1,1,11,2.720000e+10,1155-137.0,2411136,2411136,0.213
30771,30771,1534,2023-11-07,548143429,317,197,22,0.120,2,CARTON_M,2,3,CX,1155,3.0,5,1,11,2.720000e+10,1155-3.0,1373878,2747756,0.240


In [14]:
df_process_situacao1['Peso_total'].sum()

43.875

In [15]:
pecas = list(df_process_situacao1['Peças'].values)
pecas

[2,
 5,
 5,
 2,
 2,
 10,
 4,
 1,
 10,
 1,
 1,
 4,
 4,
 7,
 7,
 3,
 1,
 7,
 3,
 1,
 3,
 1,
 3,
 12,
 2,
 2,
 1,
 1,
 3,
 1,
 3,
 1,
 1,
 7,
 1,
 1,
 2,
 3,
 2,
 3,
 1,
 5,
 1,
 2,
 5,
 3,
 7,
 1,
 2,
 2,
 1,
 2,
 1,
 4,
 1,
 3,
 1,
 9,
 1,
 1,
 2,
 1]

In [16]:
peso =  list(df_process_situacao1['Peso_unit'].values)
peso

[0.284,
 0.218,
 0.45,
 0.442,
 0.315,
 0.293,
 0.16,
 0.25,
 0.2,
 0.166,
 0.166,
 0.14,
 0.222,
 0.14,
 0.14,
 0.186,
 0.216,
 0.348,
 0.37,
 0.216,
 0.49,
 0.15,
 0.11,
 0.231,
 0.261,
 0.15,
 0.11,
 0.216,
 0.11,
 0.186,
 0.185,
 0.244,
 0.2,
 0.238,
 0.174,
 0.44,
 0.73,
 0.166,
 0.11,
 0.293,
 0.173,
 0.295,
 0.19,
 0.19,
 0.175,
 0.258,
 0.22,
 0.155,
 0.168,
 0.168,
 0.313,
 0.155,
 0.213,
 0.209,
 0.213,
 0.203,
 0.213,
 0.219,
 0.15,
 0.213,
 0.12,
 0.12]

In [17]:
ids =  list(df_process_situacao1['Id'].values)
ids

[704,
 1439,
 1753,
 2191,
 2210,
 2244,
 2434,
 2484,
 2654,
 3473,
 3929,
 4071,
 4892,
 5139,
 5940,
 7427,
 7790,
 8247,
 8301,
 8465,
 8607,
 8900,
 9082,
 9475,
 10037,
 10429,
 10545,
 11418,
 12419,
 12713,
 14135,
 14643,
 15121,
 15363,
 15368,
 16320,
 17134,
 17178,
 17374,
 17382,
 17671,
 18545,
 18806,
 20472,
 20840,
 21950,
 22157,
 23199,
 23586,
 24045,
 24060,
 24515,
 27437,
 27796,
 28201,
 28466,
 29502,
 29881,
 30129,
 30672,
 30771,
 31016]

In [18]:
len(pecas)

62

In [19]:
len(peso)

62

In [20]:
# Vamos verificar se temos valores nulos no peso
any(x is None for x in peso)

False

### 1.1.2- Entrada de dados no modelo

In [21]:
ampl.param["n"] = len(pecas)
ampl.param["C"] = 23
w = peso
d = pecas

ampl.param["w"] = {i: w[i - 1] for i in range(1, len(w) + 1)}
ampl.param["d"] = {i: d[i - 1] for i in range(1, len(d) + 1)}

### 1.1.3- Solver Gurobi

In [22]:
for i in range(1, len(w) + 1):
    print(w[i - 1])

0.284
0.218
0.45
0.442
0.315
0.293
0.16
0.25
0.2
0.166
0.166
0.14
0.222
0.14
0.14
0.186
0.216
0.348
0.37
0.216
0.49
0.15
0.11
0.231
0.261
0.15
0.11
0.216
0.11
0.186
0.185
0.244
0.2
0.238
0.174
0.44
0.73
0.166
0.11
0.293
0.173
0.295
0.19
0.19
0.175
0.258
0.22
0.155
0.168
0.168
0.313
0.155
0.213
0.209
0.213
0.203
0.213
0.219
0.15
0.213
0.12
0.12


In [23]:
ampl.option["solver"] = "gurobi"
ampl.option["gurobi_options"] = "outlev=1"
ampl.solve()

Gurobi 11.0.1: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 69 rows, 441 columns and 875 nonzeros
Model fingerprint: 0x26cf4c09
Variable types: 0 continuous, 441 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 7.0000000
Presolve time: 0.00s
Presolved: 69 rows, 441 columns, 875 nonzeros
Variable types: 0 continuous, 441 integer (168 binary)

Root relaxation: objective 1.907609e+00, 104 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 

In [24]:
%%ampl_eval
display {j in J} y[j];
display {i in I, j in J} e[i,j];


y[j] [*] :=
1  0
2  0
3  0
4  0
5  0
6  1
7  1
;

e[i,j] [*,*]
:    1   2   3   4   5   6    7     :=
1    0   0   0   0   0    0    2
2    0   0   0   0   0    0    5
3    0   0   0   0   0    5    0
4    0   0   0   0   0    0    2
5    0   0   0   0   0    0    2
6    0   0   0   0   0    0   10
7    0   0   0   0   0    0    4
8    0   0   0   0   0    0    1
9    0   0   0   0   0   10    0
10   0   0   0   0   0    0    1
11   0   0   0   0   0    0    1
12   0   0   0   0   0    0    4
13   0   0   0   0   0    0    4
14   0   0   0   0   0    0    7
15   0   0   0   0   0    0    7
16   0   0   0   0   0    0    3
17   0   0   0   0   0    0    1
18   0   0   0   0   0    7    0
19   0   0   0   0   0    0    3
20   0   0   0   0   0    0    1
21   0   0   0   0   0    0    3
22   0   0   0   0   0    0    1
23   0   0   0   0   0    0    3
24   0   0   0   0   0   12    0
25   0   0   0   0   0    0    2
26   0   0   0   0   0    0    2
27   0   0   0   0   0    0    1
28   0 

In [25]:
J_set = ampl.getSet('J').getValues().toList()
y_var = ampl.getVariable('y').getValues().toPandas()
y_var

,y.val
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [26]:
e_var = ampl.getVariable('e').getValues().toPandas()
e_var

e.val
index0 index1       
1      1           0
       2           0
       3           0
       4           0
       5           0
...              ...
62     3           0
       4           0
       5           0
       6           0
       7           1

[434 rows x 1 columns]

In [27]:
e_var = e_var.reset_index()
e_var

,index0,index1,e.val
0,1,1,0
1,1,2,0
2,1,3,0
3,1,4,0
4,1,5,0
...,...,...,...
429,62,3,0
430,62,4,0
431,62,5,0
432,62,6,0


In [28]:
e_var.columns = ['itens','caixa', 'quantidade']

In [29]:
e_var

,itens,caixa,quantidade
0,1,1,0
1,1,2,0
2,1,3,0
3,1,4,0
4,1,5,0
...,...,...,...
429,62,3,0
430,62,4,0
431,62,5,0
432,62,6,0


#### Resumo do resultado

In [30]:
resultado = e_var.pivot(index='itens', columns='caixa', values='quantidade')

resultado


caixa,1,2,3,4,5,6,7
itens,,,,,,,
1,0,0,0,0,0,0,2
2,0,0,0,0,0,0,5
3,0,0,0,0,0,5,0
4,0,0,0,0,0,0,2
5,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...
58,0,0,0,0,0,9,0
59,0,0,0,0,0,0,1
60,0,0,0,0,0,0,1


In [31]:
resultado = resultado.reset_index()
resultado.columns.name = None
resultado['id'] = ids

resultado

,itens,1,2,3,4,5,6,7,id
0,1,0,0,0,0,0,0,2,704
1,2,0,0,0,0,0,0,5,1439
2,3,0,0,0,0,0,5,0,1753
3,4,0,0,0,0,0,0,2,2191
4,5,0,0,0,0,0,0,2,2210
...,...,...,...,...,...,...,...,...,...
57,58,0,0,0,0,0,9,0,29881
58,59,0,0,0,0,0,0,1,30129
59,60,0,0,0,0,0,0,1,30672
60,61,0,0,0,0,0,2,0,30771


In [32]:
resultado.columns

Index(['itens', 1, 2, 3, 4, 5, 6, 7, 'id'], dtype='object')

In [33]:
resultado_final_s1 = resultado.loc[:, (resultado != 0).any(axis=0)]
with pd.option_context('display.max_rows', None):
    display(resultado_final_s1)

,itens,6,7,id
0,1,0,2,704
1,2,0,5,1439
2,3,5,0,1753
3,4,0,2,2191
4,5,0,2,2210
5,6,0,10,2244
6,7,0,4,2434
7,8,0,1,2484
8,9,10,0,2654
9,10,0,1,3473


In [34]:
resultado_final_s1.columns

Index(['itens', 6, 7, 'id'], dtype='object')

In [35]:
print(f"número de caixas necessárias: {len(resultado_final_s1.drop(columns=['itens','id']).columns)}")

número de caixas necessárias: 2


In [36]:
for i in resultado_final_s1[[6,7]].columns.values:
    print(f"quantidade itens na mochila {i}: {resultado_final_s1[i].sum()}")

quantidade itens na mochila 6: 88
quantidade itens na mochila 7: 101


Com as restrições báscias, todos os itens foram colocados nas caixas

## 1.2- CSP com problemas do negócio - versão 1

Refazendo a modelagem

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, cada um com peso $w_{i}$, um volume $v_{i}$, um múltiplo $inner_{i}$ da quantiade de itens a serem selecioner uma demanda $d_{i}$, para todo $i \in \{1,...,n\}$, e um número $m$ de mochilas (caixas) idênticas com capacidade volumétrica $cv$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} y_{j} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i} \xi_{i j}*inner_{i} \leq cp*y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{i = 1}^{n} v_{i} \xi_{i j} *inner_{i} \leq cv*y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{i = 1}^{n} \xi_{i j} *inner_{i} \leq np*y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{j = 1}^{m} \xi_{i j} *inner_{i} = d_{i} \quad \forall i \in \{1,...,n\} \\
&\xi_{i j} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} \\
&y_{j} \in \{0,1\} \quad \forall j \in \{1,...,m\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j}$= número de itens tipo $i$ adicionados na mochila $j$


In [37]:
%%ampl_eval
reset;

In [38]:
%%ampl_eval
param n;
param cp;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
param w {I} > 0;
param inner{I} > 0;
param d {I} > 0;
param maxVal := max {i in I} w[i];
param soma := sum{i in I} d[i];
param maxbins := ceil(soma / floor(cp / maxVal));

set J = 1..maxbins;

var e {I,J} integer >= 0;
var y {J} binary;

minimize FO:  sum {j in J} y[j];

s.t. R1_capacidade_mochila_peso {j in J}:
   sum {i in I} w[i] * e[i,j] * inner[i] <= cp * y[j] suffix block j;

s.t. R4_quantidade_demanda {i in I}:
   sum {j in J} e[i, j]* inner[i] = d[i] suffix block i;


### 1.2.1- Importar dados

In [39]:
df_caixas_tipo = pd.read_excel('../01-Data/Dados - desafio disciplina RPVMM.xlsx', sheet_name='Tipos de caixa')
df_caixas_tipo

,Tipo,Tipo de buffer,Comprimento,Largura,Altura,Fator de ocupação,Peças max,Peso max (kg)
0,Caixa 1,CX,380,285,285,0.9,170,23
1,Caixa 2,CX,586,436,272,1.2,170,23
2,Caixa 3,CX,728,427,270,0.9,170,23
3,Caixa 1,EN,380,285,285,0.9,512,23
4,Caixa 2,EN,586,436,272,1.2,512,23
5,Caixa 3,EN,728,427,270,0.9,512,23


In [40]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')

In [41]:
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']

df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


Temos um tipo de caixa padrão para cada tipo de buffer

In [42]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [43]:
df_caixas_padrao[['Tipo','Tipo_de_buffer']] = df_caixas_padrao[['Tipo','Tipo_de_buffer']].astype('string')

In [44]:
df_caixas_padrao.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 1 to 5
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tipo               2 non-null      string 
 1   Tipo_de_buffer     2 non-null      string 
 2   Comprimento        2 non-null      int64  
 3   Largura            2 non-null      int64  
 4   Altura             2 non-null      int64  
 5   Fator_de_ocupação  2 non-null      float64
 6   Peças_max          2 non-null      int64  
 7   Peso_max_(kg)      2 non-null      int64  
 8   volume_util_caixa  2 non-null      float64
dtypes: float64(2), int64(5), string(2)
memory usage: 160.0 bytes


In [45]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58458 entries, 0 to 58457
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         58458 non-null  int64         
 1   Ordem_de_produção          58458 non-null  int64         
 2   Data_da_ordem_de_produção  58458 non-null  datetime64[ns]
 3   Produto                    58458 non-null  int64         
 4   Comprimento_unit           58458 non-null  int64         
 5   Largura_unit               58458 non-null  int64         
 6   Altura_unit                58458 non-null  int64         
 7   Peso_unit                  58458 non-null  float64       
 8   Peças                      58458 non-null  int64         
 9   Caixa_padrão               58458 non-null  object        
 10  Loja                       58458 non-null  string        
 11  Classe_de_onda             58458 non-null  string        
 12  Tipo

In [46]:
df_process[['Loja','Tipo_de_buffer','Classe_de_onda']] = df_process[['Loja','Tipo_de_buffer','Classe_de_onda']].astype('string')

In [47]:
df_process['chave_loja_buffer_onda'] = df_process['Loja'] + '-' + df_process['Tipo_de_buffer'] + '-' + df_process['Classe_de_onda']
df_process['chave_loja_buffer_onda'] 

0          2-CX-2
1          3-CX-1
2          6-CX-6
3          7-CX-4
4          2-CX-4
           ...   
58453      9-EN-3
58454    149-EN-2
58455     50-EN-4
58456     82-EN-2
58457     11-EN-1
Name: chave_loja_buffer_onda, Length: 58458, dtype: string

In [48]:
df_process.groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(180)

chave_loja_buffer_onda
10-EN-3    2470
28-EN-4    2392
16-EN-3    1636
2-EN-3     1526
8-EN-4     1369
           ... 
10-CX-5     336
51-EN-1     335
64-CX-4     329
2-CX-6      326
30-EN-3     325
Name: Peças, Length: 180, dtype: int64

In [49]:
df_process_situacao2 = df_process.query('chave_loja_buffer_onda == "2-CX-3"')
df_process_situacao2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade,pai-cor,Volume_unit,Volume_total,Peso_total,chave_loja_buffer_onda
704,704,1237,2023-11-16,736849445,447,268,20,0.284,2,CARTON_M,...,347.0,6,1,11,2.720000e+10,992-347.0,2395920,4791840,0.568,2-CX-3
1439,1439,59,2023-11-14,841264182,437,192,16,0.218,5,CARTON_M,...,37.0,3,1,11,2.720000e+10,59-37.0,1342464,6712320,1.090,2-CX-3
1753,1753,987,2023-11-14,882057278,449,239,62,0.450,5,CARTON_M,...,9.0,5,1,11,2.720000e+10,827-9.0,6653282,33266410,2.250,2-CX-3
2191,2191,933,2023-11-14,856853019,564,277,24,0.442,2,CARTON_M,...,4.0,6,1,11,2.720000e+10,792-4.0,3749472,7498944,0.884,2-CX-3
2210,2210,989,2023-11-14,675645211,407,226,27,0.315,2,CARTON_M,...,4.0,1,1,11,2.720000e+10,829-4.0,2483514,4967028,0.630,2-CX-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29881,29881,7585,2023-11-14,841264123,446,201,17,0.219,9,CARTON_M,...,4.0,3,1,11,2.720000e+10,59-4.0,1523982,13715838,1.971,2-CX-3
30129,30129,1723,2023-11-07,595813106,335,172,38,0.150,1,CARTON_M,...,77.0,3,1,11,2.720000e+10,503-77.0,2189560,2189560,0.150,2-CX-3
30672,30672,1534,2023-11-07,817850042,364,288,23,0.213,1,CARTON_M,...,137.0,1,1,11,2.720000e+10,1155-137.0,2411136,2411136,0.213,2-CX-3
30771,30771,1534,2023-11-07,548143429,317,197,22,0.120,2,CARTON_M,...,3.0,5,1,11,2.720000e+10,1155-3.0,1373878,2747756,0.240,2-CX-3


In [50]:
df_process_situacao2['Peças'].sum()

189

In [51]:
df_process_situacao2.describe().T

,count,mean,min,25%,50%,75%,max,std
Id,62.0,14566.887097,704.0,7517.75,14389.0,22105.25,31016.0,9360.050105
Ordem_de_produção,62.0,2853.306452,59.0,1534.0,1870.5,3774.5,7688.0,1930.181564
Data_da_ordem_de_produção,62,2023-11-11 06:58:03.870967808,2023-11-07 00:00:00,2023-11-07 00:00:00,2023-11-07 00:00:00,2023-11-14 00:00:00,2023-11-27 00:00:00,NaN
Produto,62.0,711792870.145161,519980324.0,555234635.0,742248804.0,844408471.0,885051605.0,142475093.610776
Comprimento_unit,62.0,369.145161,237.0,317.0,349.5,420.25,564.0,69.3215
Largura_unit,62.0,237.112903,169.0,198.75,229.5,279.5,327.0,46.50722
Altura_unit,62.0,24.709677,10.0,16.25,23.0,27.75,62.0,11.403748
Peso_unit,62.0,0.226597,0.11,0.166,0.206,0.2485,0.73,0.107469
Peças,62.0,3.048387,1.0,1.0,2.0,4.0,12.0,2.620396
Item_pai,62.0,834.483871,19.0,503.0,809.5,1155.0,2367.0,522.08437


In [52]:
df_process_situacao2.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade', 'pai-cor',
       'Volume_unit', 'Volume_total', 'Peso_total', 'chave_loja_buffer_onda'],
      dtype='object')

In [53]:
df_process_situacao2['Tipo_de_buffer'].unique()

<StringArray>
['CX']
Length: 1, dtype: string

### 1.2.2- Entrada de dados no modelo

In [54]:
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


Tipo de buffer da lista selecionada:

In [55]:
df_process_situacao2['Tipo_de_buffer'].unique()[0]

'CX'

In [56]:
peso =  list(df_process_situacao2['Peso_unit'].values)
pecas = list(df_process_situacao2['Peças'].values)
volume = list(df_process_situacao2['Volume_unit'].values)
inner = list(df_process_situacao2['Inner'].values)

In [57]:
buffer = df_process_situacao2['Tipo_de_buffer'].unique()[0]
cp = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['Peso_max_(kg)'].values

print(cp)

[23]


In [58]:
ampl.param["n"] = len(pecas)

ampl.param["cp"] = cp
# ampl.param["cv"] = cv
# ampl.param["np"] = np

w = peso
v = volume
d = pecas
inner = inner

ampl.param["w"] = {i: w[i - 1] for i in range(1, len(w) + 1)}
# ampl.param["v"] = {i: v[i - 1] for i in range(1, len(v) + 1)}
ampl.param["inner"] = {i: inner[i - 1] for i in range(1, len(inner) + 1)}
ampl.param["d"] = {i: d[i - 1] for i in range(1, len(d) + 1)}

In [59]:
ampl.getValue('maxbins')

7

In [60]:
ampl.getValue('soma')

189

In [61]:
ampl.getValue('maxVal')

0.73

In [62]:
any(x is None for x in peso)

False

### 1.2.3- Solver Gurobi

In [63]:
ampl.option["solver"] = "gurobi"
ampl.option["gurobi_options"] = "outlev=1"
ampl.solve()

Gurobi 11.0.1: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 69 rows, 441 columns and 875 nonzeros
Model fingerprint: 0x26cf4c09
Variable types: 0 continuous, 441 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 7.0000000
Presolve time: 0.00s
Presolved: 69 rows, 441 columns, 875 nonzeros
Variable types: 0 continuous, 441 integer (168 binary)

Root relaxation: objective 1.907609e+00, 104 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 

In [64]:
%%ampl_eval
display {j in J} y[j];
display {i in I, j in J} e[i,j];

y[j] [*] :=
1  0
2  0
3  0
4  0
5  0
6  1
7  1
;

e[i,j] [*,*]
:    1   2   3   4   5   6    7     :=
1    0   0   0   0   0    0    2
2    0   0   0   0   0    0    5
3    0   0   0   0   0    5    0
4    0   0   0   0   0    0    2
5    0   0   0   0   0    0    2
6    0   0   0   0   0    0   10
7    0   0   0   0   0    0    4
8    0   0   0   0   0    0    1
9    0   0   0   0   0   10    0
10   0   0   0   0   0    0    1
11   0   0   0   0   0    0    1
12   0   0   0   0   0    0    4
13   0   0   0   0   0    0    4
14   0   0   0   0   0    0    7
15   0   0   0   0   0    0    7
16   0   0   0   0   0    0    3
17   0   0   0   0   0    0    1
18   0   0   0   0   0    7    0
19   0   0   0   0   0    0    3
20   0   0   0   0   0    0    1
21   0   0   0   0   0    0    3
22   0   0   0   0   0    0    1
23   0   0   0   0   0    0    3
24   0   0   0   0   0   12    0
25   0   0   0   0   0    0    2
26   0   0   0   0   0    0    2
27   0   0   0   0   0    0    1
28   0 

In [65]:
e_var = ampl.getVariable('e').getValues().toPandas()
e_var = e_var.reset_index()
e_var.columns = ['itens','caixa', 'quantidade']

e_var

,itens,caixa,quantidade
0,1,1,0
1,1,2,0
2,1,3,0
3,1,4,0
4,1,5,0
...,...,...,...
429,62,3,0
430,62,4,0
431,62,5,0
432,62,6,0


In [66]:
resultado = e_var.pivot(index='itens', columns='caixa', values='quantidade')

resultado


caixa,1,2,3,4,5,6,7
itens,,,,,,,
1,0,0,0,0,0,0,2
2,0,0,0,0,0,0,5
3,0,0,0,0,0,5,0
4,0,0,0,0,0,0,2
5,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...
58,0,0,0,0,0,9,0
59,0,0,0,0,0,0,1
60,0,0,0,0,0,0,1


In [67]:
y_var = ampl.getVariable('y').getValues().toPandas()
y_var

,y.val
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [68]:
resultado_final_s2 = resultado.loc[:, (resultado != 0).any(axis=0)]
with pd.option_context('display.max_rows', None):
    display(resultado_final_s2)

caixa,6,7
itens,,
1,0,2
2,0,5
3,5,0
4,0,2
5,0,2
6,0,10
7,0,4
8,0,1
9,10,0


In [69]:
for i in resultado_final_s2[[6,7]].columns.values:
    print(f"quantidade itens na mochila {i}: {resultado_final_s2[i].sum()}")

quantidade itens na mochila 6: 88
quantidade itens na mochila 7: 101


### 1.2.4- Outros solvers

#### Solver GCG

In [70]:
ampl.option["solver"] = "gcg"
ampl.option["gcg_options"] = "outlev=1"
ampl.solve()

GCG 4.0.0:   tech:outlev = 1
 added complete decomp for original problem with 62 blocks and 0 masterconss, 414 linkingvars, 0 mastervars, and max white score of   0.030826 
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 23 upgd conss, 0 impls, 23 clqs
   (0.0s) probing cycle finished: starting next cycle
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 184 cliques
presolved problem has 441 variables (168 bin, 273 int, 0 impl, 0 cont) and 69 constraints
     23 constraints of type <setppc>
     46 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00
 calculated translation; number of missing constraints: 0; number of other partialdecs: 1 
Preexisting decomposition found. Solution process started...

A Dantzig-W

In [71]:
%%ampl_eval
display {j in J} y[j];
display {i in I, j in J} e[i,j];

y[j] [*] :=
1  1
2  1
3  0
4  0
5  0
6  0
7  0
;

e[i,j] [*,*]
:    1    2    3   4   5   6   7    :=
1     2    0   0   0   0   0   0
2     5    0   0   0   0   0   0
3     5    0   0   0   0   0   0
4     2    0   0   0   0   0   0
5     2    0   0   0   0   0   0
6    10    0   0   0   0   0   0
7     4    0   0   0   0   0   0
8     1    0   0   0   0   0   0
9    10    0   0   0   0   0   0
10    1    0   0   0   0   0   0
11    1    0   0   0   0   0   0
12    4    0   0   0   0   0   0
13    4    0   0   0   0   0   0
14    7    0   0   0   0   0   0
15    7    0   0   0   0   0   0
16    3    0   0   0   0   0   0
17    1    0   0   0   0   0   0
18    7    0   0   0   0   0   0
19    0    3   0   0   0   0   0
20    1    0   0   0   0   0   0
21    0    3   0   0   0   0   0
22    1    0   0   0   0   0   0
23    3    0   0   0   0   0   0
24    0   12   0   0   0   0   0
25    2    0   0   0   0   0   0
26    0    2   0   0   0   0   0
27    1    0   0   0   0   0   0
28    1

In [72]:
e_var_GCG = ampl.getVariable('e').getValues().toPandas()
e_var_GCG = e_var_GCG.reset_index()
e_var_GCG.columns = ['itens','caixa', 'quantidade']

resultado_GCG = e_var_GCG.pivot(index='itens', columns='caixa', values='quantidade')
resultado_GCG

caixa,1,2,3,4,5,6,7
itens,,,,,,,
1,2,0,0,0,0,0,0
2,5,0,0,0,0,0,0
3,5,0,0,0,0,0,0
4,2,0,0,0,0,0,0
5,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...
58,0,9,0,0,0,0,0
59,1,0,0,0,0,0,0
60,1,0,0,0,0,0,0


In [73]:
resultado_GCG = resultado_GCG.reset_index()
resultado_GCG.columns.name = None
resultado_GCG['id'] = ids

resultado_GCG

,itens,1,2,3,4,5,6,7,id
0,1,2,0,0,0,0,0,0,704
1,2,5,0,0,0,0,0,0,1439
2,3,5,0,0,0,0,0,0,1753
3,4,2,0,0,0,0,0,0,2191
4,5,2,0,0,0,0,0,0,2210
...,...,...,...,...,...,...,...,...,...
57,58,0,9,0,0,0,0,0,29881
58,59,1,0,0,0,0,0,0,30129
59,60,1,0,0,0,0,0,0,30672
60,61,0,2,0,0,0,0,0,30771


In [74]:
resultado_final_s2_GCG = resultado_GCG.loc[:, (resultado_GCG != 0).any(axis=0)]
with pd.option_context('display.max_rows', None):
    display(resultado_final_s2_GCG)

,itens,1,2,id
0,1,2,0,704
1,2,5,0,1439
2,3,5,0,1753
3,4,2,0,2191
4,5,2,0,2210
5,6,10,0,2244
6,7,4,0,2434
7,8,1,0,2484
8,9,10,0,2654
9,10,1,0,3473


In [75]:
for i in resultado_final_s2_GCG.drop(columns=['itens', 'id']).columns.values:
    print(f"quantidade itens na mochila {i}: {resultado_final_s2_GCG[i].sum()}")

quantidade itens na mochila 1: 100
quantidade itens na mochila 2: 89


#### Solver HIGHS

In [76]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-01, 2e+01]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+01]
  RHS    [1e+00, 1e+01]
Assessing feasibility of MIP using primal feasibility and integrality tolerance of       1e-06
Solution has               num          max          sum
Col     infeasibilities      0            0            0
Integer infeasibilities      0            0            0
Row     infeasibilities      0            0            0
Row     residuals            0            0            0
Presolving model
69 rows, 441 cols, 875 nonzeros  0s
69 rows, 441 cols, 875 nonzeros  0s

MIP start solution is feasible, objective value is 2
Objective function is integral with scale 1

Solving MIP model with:
   69 rows
   441 cols (168 binary, 273 integer, 0 implied int., 0 continuous)
   875 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds         

In [77]:
%%ampl_eval
display {j in J} y[j];
display {i in I, j in J} e[i,j];

y[j] [*] :=
1  1
2  1
3  0
4  0
5  0
6  0
7  0
;

e[i,j] [*,*]
:    1    2    3   4   5   6   7    :=
1     2    0   0   0   0   0   0
2     5    0   0   0   0   0   0
3     5    0   0   0   0   0   0
4     2    0   0   0   0   0   0
5     2    0   0   0   0   0   0
6    10    0   0   0   0   0   0
7     4    0   0   0   0   0   0
8     1    0   0   0   0   0   0
9    10    0   0   0   0   0   0
10    1    0   0   0   0   0   0
11    1    0   0   0   0   0   0
12    4    0   0   0   0   0   0
13    4    0   0   0   0   0   0
14    7    0   0   0   0   0   0
15    7    0   0   0   0   0   0
16    3    0   0   0   0   0   0
17    1    0   0   0   0   0   0
18    7    0   0   0   0   0   0
19    0    3   0   0   0   0   0
20    1    0   0   0   0   0   0
21    0    3   0   0   0   0   0
22    1    0   0   0   0   0   0
23    3    0   0   0   0   0   0
24    0   12   0   0   0   0   0
25    2    0   0   0   0   0   0
26    0    2   0   0   0   0   0
27    1    0   0   0   0   0   0
28    1

In [78]:
e_var_highs = ampl.getVariable('e').getValues().toPandas()
e_var_highs = e_var_highs.reset_index()
e_var_highs.columns = ['itens','caixa', 'quantidade']

resultado_highs = e_var_highs.pivot(index='itens', columns='caixa', values='quantidade')
resultado_highs

caixa,1,2,3,4,5,6,7
itens,,,,,,,
1,2,0,0,0,0,0,0
2,5,0,0,0,0,0,0
3,5,0,0,0,0,0,0
4,2,0,0,0,0,0,0
5,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...
58,0,9,0,0,0,0,0
59,1,0,0,0,0,0,0
60,1,0,0,0,0,0,0


In [79]:
resultado_highs = resultado_highs.reset_index()
resultado_highs.columns.name = None
resultado_highs['id'] = ids

resultado_final_s2_highs = resultado_highs.loc[:, (resultado_highs != 0).any(axis=0)]
with pd.option_context('display.max_rows', None):
    display(resultado_final_s2_highs)

,itens,1,2,id
0,1,2,0,704
1,2,5,0,1439
2,3,5,0,1753
3,4,2,0,2191
4,5,2,0,2210
5,6,10,0,2244
6,7,4,0,2434
7,8,1,0,2484
8,9,10,0,2654
9,10,1,0,3473


In [80]:
resultado_final_s2_highs.columns

Index(['itens', 1, 2, 'id'], dtype='object')

In [81]:
for i in resultado_final_s2_highs.drop(columns=['itens', 'id']).columns.values:
    print(f"quantidade itens na mochila {i}: {resultado_final_s2_highs[i].sum()}")

quantidade itens na mochila 1: 100
quantidade itens na mochila 2: 89


## 1.3- CSP com problemas do negócio - versão 2

Será adicionada a restrição de número máximo de peças por caixa.

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, cada um com peso $w_{i}$ e um múltiplo $inner_{i}$ da quantiade de itens a serem selecioner uma demanda $d_{i}$, para todo $i \in \{1,...,n\}$, e um número $m$ de mochilas (caixas) idênticas com capacidade volumétrica $cv$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} y_{j} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i} \xi_{i j}*inner_{i} \leq cp*y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{i = 1}^{n} \xi_{i j} *inner_{i} \leq np*y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{j = 1}^{m} \xi_{i j} *inner_{i} = d_{i} \quad \forall i \in \{1,...,n\} \\
&\xi_{i j} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} \\
&y_{j} \in \{0,1\} \quad \forall j \in \{1,...,m\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j}$= número de itens tipo $i$ adicionados na mochila $j$


In [82]:
%%ampl_eval
reset;

In [83]:
%%ampl_eval
param n;
param cp;
param np;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
param w {I} > 0;
param inner{I} > 0;
param d {I} > 0;
param maxVal := max {i in I} w[i];
param soma := sum{i in I} d[i];
param maxbins := ceil(soma / floor(cp / maxVal));

set J = 1..maxbins;

var e {I,J} integer >= 0;
var y {J} binary;

minimize FO:  sum {j in J} y[j];

s.t. R1_capacidade_mochila_peso {j in J}:
   sum {i in I} w[i] * e[i,j] * inner[i] <= cp * y[j] suffix block j;

s.t. R3_capacidade_mochila_pecas {j in J}:
   sum {i in I} e[i,j] * inner[i] <= np * y[j] suffix block j;

s.t. R4_quantidade_demanda {i in I}:
   sum {j in J} e[i, j]* inner[i] = d[i] suffix block i;


### 1.3.1- Importar dados

In [84]:
df_process_situacao3 = df_process.query('chave_loja_buffer_onda == "2-CX-9"')
df_process_situacao3

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade,pai-cor,Volume_unit,Volume_total,Peso_total,chave_loja_buffer_onda
149,149,277,2023-11-14,768650318,300,172,89,0.260,24,CARTON_M,...,54.0,12,4,11,2.720000e+10,268-54.0,4592400,110217600,6.240,2-CX-9
221,221,407,2023-11-11,807049518,448,196,24,0.215,8,CARTON_M,...,18.0,6,2,11,2.720000e+10,391-18.0,2107392,16859136,1.720,2-CX-9
590,590,1027,2023-11-07,883874958,440,191,19,0.113,5,CARTON_M,...,124.0,5,1,11,2.720000e+10,433-124.0,1596760,7983800,0.565,2-CX-9
670,670,1172,2023-11-14,552909458,300,297,30,0.220,1,CARTON_M,...,3.0,3,1,11,2.720000e+10,151-3.0,2673000,2673000,0.220,2-CX-9
763,763,1172,2023-11-03,542140534,300,259,25,0.230,5,CARTON_M,...,74.0,1,1,11,2.720000e+10,135-74.0,1942500,9712500,1.150,2-CX-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29581,29581,6690,2023-11-27,739849299,314,198,110,0.840,2,CARTON_M,...,53.0,52,1,11,2.720000e+10,360-53.0,6838920,13677840,1.680,2-CX-9
29690,29690,7034,2023-11-13,597109315,330,201,128,0.700,2,CARTON_M,...,42.0,18,1,11,2.720000e+10,1359-42.0,8490240,16980480,1.400,2-CX-9
29901,29901,6967,2023-11-14,625821175,300,174,12,0.078,32,CARTON_M,...,18.0,17,4,11,2.720000e+10,2354-18.0,626400,20044800,2.496,2-CX-9
30685,30685,7169,2023-11-13,550943311,325,201,116,0.620,3,CARTON_M,...,306.0,21,1,11,2.720000e+10,865-306.0,7577700,22733100,1.860,2-CX-9


### 1.3.2- Entrada de dados no modelo

In [85]:
peso =  list(df_process_situacao3['Peso_unit'].values)
pecas = list(df_process_situacao3['Peças'].values)
inner = list(df_process_situacao3['Inner'].values)

In [86]:
buffer = df_process_situacao3['Tipo_de_buffer'].unique()[0]
cp = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['Peso_max_(kg)'].values
np = 170  #reduzida a capacidade de peças para alterar os resultados do solver
print(cp, np)

[23] 170


In [87]:
ampl.param["n"] = len(pecas)

ampl.param["cp"] = cp
ampl.param["np"] = np

w = peso
d = pecas
inner = inner

ampl.param["w"] = {i: w[i - 1] for i in range(1, len(w) + 1)}
ampl.param["inner"] = {i: inner[i - 1] for i in range(1, len(inner) + 1)}
ampl.param["d"] = {i: d[i - 1] for i in range(1, len(d) + 1)}

verificações

In [88]:
ampl.getValue('cp')

23

In [89]:
ampl.getValue('maxVal')

0.98

In [90]:
ampl.getValue('soma')

913

In [91]:
ampl.getValue('maxbins')

40

In [92]:
ids =  list(df_process_situacao3['Id'].values)
ids

[149,
 221,
 590,
 670,
 763,
 788,
 879,
 1391,
 1447,
 1572,
 1694,
 1884,
 2041,
 2302,
 2382,
 2466,
 2755,
 3729,
 3790,
 3793,
 3944,
 5396,
 5452,
 7142,
 7342,
 7540,
 7831,
 7896,
 7914,
 8244,
 8311,
 8650,
 8684,
 8704,
 8866,
 9060,
 9073,
 9083,
 9099,
 9144,
 9498,
 9656,
 9775,
 9900,
 10311,
 10312,
 10324,
 10409,
 10493,
 11093,
 11098,
 11419,
 11555,
 13723,
 14099,
 14609,
 14709,
 14846,
 15665,
 15686,
 15790,
 15879,
 16301,
 16315,
 16443,
 16777,
 16846,
 16915,
 16922,
 17237,
 18577,
 18950,
 19531,
 20568,
 21204,
 21367,
 21686,
 21874,
 21895,
 22215,
 22216,
 22380,
 22571,
 22618,
 22768,
 22818,
 22977,
 22990,
 23009,
 23381,
 24353,
 24475,
 24611,
 24898,
 24996,
 25120,
 26031,
 27238,
 27267,
 27502,
 27791,
 27900,
 27920,
 28244,
 29072,
 29077,
 29442,
 29581,
 29690,
 29901,
 30685,
 31430]

### 1.3.3 Solver Gurobi

In [93]:
ampl.option["solver"] = "gurobi"
ampl.option["gurobi_options"] = "outlev=1"
ampl.solve()

Gurobi 11.0.1: Set parameter LogToConsole to value 1
  tech:outlev = 1

Sorry, a demo license is limited to 2000 variables and
2000 constraints and objectives for linear problems.
You have 4520 variables and 192 constraints.

Contact us at <licensing@ampl.com> or go to https://discuss.ampl.com/
for more information about licensing options.
exit value 1
<BREAK>


In [94]:
%%ampl_eval
display {j in J} y[j];
display {i in I, j in J} e[i,j];

y[j] [*] :=
 1 0    5 0    9 0   13 0   17 0   21 0   25 0   29 0   33 0   37 0
 2 0    6 0   10 0   14 0   18 0   22 0   26 0   30 0   34 0   38 0
 3 0    7 0   11 0   15 0   19 0   23 0   27 0   31 0   35 0   39 0
 4 0    8 0   12 0   16 0   20 0   24 0   28 0   32 0   36 0   40 0
;

e[i,j] [*,*]
:     1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 :=
1     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
2     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
3     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
4     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
5     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
6     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
7     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
8     0   0   0   0   0   0   0   0   0   0   0   0   0  

In [95]:
e_var_s3 = ampl.getVariable('e').getValues().toPandas()
e_var_s3 = e_var_s3.reset_index()
e_var_s3.columns = ['itens','caixa', 'quantidade']

resultado_s3 = e_var_s3.pivot(index='itens', columns='caixa', values='quantidade')


resultado_s3 = resultado_s3.reset_index()
resultado_s3.columns.name = None
resultado_s3['id'] = ids

resultado_s3

,itens,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,149
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,221
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,590
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,670
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,108,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,29581
108,109,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,29690
109,110,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,29901
110,111,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30685


In [96]:
resultado_final_s3 = resultado_s3.loc[:, (resultado_s3 != 0).any(axis=0)]
with pd.option_context('display.max_rows', None):
    display(resultado_final_s3)

,itens,id
0,1,149
1,2,221
2,3,590
3,4,670
4,5,763
5,6,788
6,7,879
7,8,1391
8,9,1447
9,10,1572


In [97]:
resultado_final_s3.drop(columns=['itens', 'id'])

""
0
1
2
3
4
...
107
108
109
110


Resumo

In [98]:
for i in resultado_final_s3.drop(columns=['itens', 'id']).columns.values:
    print(f"quantidade de itens na mochila {i}: {resultado_final_s3[i].sum()}")

### 1.3.4- Outros Solvers

#### Solver GCG

In [99]:
ampl.option["solver"] = "gcg"
ampl.option["gcg_options"] = "outlev=1"
ampl.solve()

GCG 4.0.0:   tech:outlev = 1
 added complete decomp for original problem with 112 blocks and 0 masterconss, 4361 linkingvars, 0 mastervars, and max white score of   0.017423 
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 21 upgd conss, 0 impls, 21 clqs
   (0.0s) probing cycle finished: starting next cycle
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 861 cliques
presolved problem has 4520 variables (880 bin, 3640 int, 0 impl, 0 cont) and 192 constraints
     21 constraints of type <setppc>
    171 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00
 calculated translation; number of missing constraints: 0; number of other partialdecs: 1 
Preexisting decomposition found. Solution process started...

A Dant

In [100]:
%%ampl_eval
display {j in J} y[j];
display {i in I, j in J} e[i,j];

y[j] [*] :=
 1 1    5 1    9 0   13 0   17 0   21 0   25 0   29 0   33 0   37 0
 2 1    6 1   10 0   14 0   18 0   22 0   26 0   30 0   34 0   38 0
 3 1    7 1   11 0   15 0   19 0   23 0   27 0   31 0   35 0   39 0
 4 1    8 1   12 0   16 0   20 0   24 0   28 0   32 0   36 0   40 0
;

e[i,j] [*,*]
:     1   2    3    4    5    6    7    8    9  10  11  12  13  14  15  16  17 :=
1     6    0    0    0    0    0    0    0   0   0   0   0   0   0   0   0   0
2     4    0    0    0    0    0    0    0   0   0   0   0   0   0   0   0   0
3     5    0    0    0    0    0    0    0   0   0   0   0   0   0   0   0   0
4     1    0    0    0    0    0    0    0   0   0   0   0   0   0   0   0   0
5     5    0    0    0    0    0    0    0   0   0   0   0   0   0   0   0   0
6     0    5    0    0    0    0    0    0   0   0   0   0   0   0   0   0   0
7     0   12    0    0    0    0    0    0   0   0   0   0   0   0   0   0   0
8     0    7    0    0    0    0    0    0   0   0   0   0   0   

In [101]:
e_var_s3_GCG = ampl.getVariable('e').getValues().toPandas()
e_var_s3_GCG = e_var_s3_GCG.reset_index()
e_var_s3_GCG.columns = ['itens','caixa', 'quantidade']

resultado_s3_GCG = e_var_s3_GCG.pivot(index='itens', columns='caixa', values='quantidade')


resultado_s3_GCG = resultado_s3_GCG.reset_index()
resultado_s3_GCG.columns.name = None
resultado_s3_GCG['id'] = ids
resultado_s3_GCG['pecas'] = pecas
resultado_s3_GCG['inner'] = inner

resultado_s3_GCG

,itens,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,id,pecas,inner
0,1,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,149,24,4
1,2,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,221,8,2
2,3,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,590,5,1
3,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,670,1,1
4,5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,763,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,108,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,29581,2,1
108,109,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,29690,2,1
109,110,0,0,0,0,0,0,0,8,0,...,0,0,0,0,0,0,0,29901,32,4
110,111,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,30685,3,1


In [102]:
resultado_final_s3_GCG = resultado_s3_GCG.loc[:, (resultado_s3_GCG != 0).any(axis=0)]
with pd.option_context('display.max_rows', None):
    display(resultado_final_s3_GCG)

,itens,1,2,3,4,5,6,7,8,id,pecas,inner
0,1,6,0,0,0,0,0,0,0,149,24,4
1,2,4,0,0,0,0,0,0,0,221,8,2
2,3,5,0,0,0,0,0,0,0,590,5,1
3,4,1,0,0,0,0,0,0,0,670,1,1
4,5,5,0,0,0,0,0,0,0,763,5,1
5,6,0,5,0,0,0,0,0,0,788,10,2
6,7,0,12,0,0,0,0,0,0,879,12,1
7,8,0,7,0,0,0,0,0,0,1391,21,3
8,9,0,2,0,0,0,0,0,0,1447,2,1
9,10,0,3,0,0,0,0,0,0,1572,3,1


In [103]:
resultado_final_s3_GCG['soma_multiplo'] = resultado_final_s3_GCG['pecas']/resultado_final_s3_GCG['inner']
resultado_final_s3_GCG['soma_multiplo']

0       6.0
1       4.0
2       5.0
3       1.0
4       5.0
       ... 
107     2.0
108     2.0
109     8.0
110     3.0
111    16.0
Name: soma_multiplo, Length: 112, dtype: float64

Verificação dos resultados

In [104]:
for i in resultado_final_s3_GCG.drop(columns=['itens', 'id', 'inner', 'pecas']).columns.values:
    print(f"soma de múltiplo {i}: {resultado_final_s3_GCG[i].sum()}")

soma de múltiplo 1: 43
soma de múltiplo 2: 82
soma de múltiplo 3: 61
soma de múltiplo 4: 79
soma de múltiplo 5: 84
soma de múltiplo 6: 102
soma de múltiplo 7: 62
soma de múltiplo 8: 55
soma de múltiplo soma_multiplo: 568.0


In [105]:
total_soma_resultado = 0

for i in resultado_final_s3_GCG.drop(columns=['itens', 'id', 'inner', 'pecas', 'soma_multiplo']).columns.values:
    total_soma_resultado += resultado_final_s3_GCG[i].sum()

print(total_soma_resultado)

568


In [106]:
print(f"quantidade de itens encaixotados: {resultado_final_s3_GCG['pecas'].sum()}")

quantidade de itens encaixotados: 913


## 1.4- CSP com problemas do negócio - versão 3

Vamos adicionar a restrição de volume de cada caixa

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, cada um com peso $w_{i}$, um volume $v_{i}$, um múltiplo $inner_{i}$ da quantiade de itens a serem selecioner uma demanda $d_{i}$, para todo $i \in \{1,...,n\}$, e um número $m$ de mochilas (caixas) idênticas com capacidade volumétrica $cv$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} y_{j} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i} \xi_{i j}*inner_{i} \leq cp*y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{i = 1}^{n} v_{i} \xi_{i j} *inner_{i} \leq cv*y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{i = 1}^{n} \xi_{i j} *inner_{i} \leq np*y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{j = 1}^{m} \xi_{i j} *inner_{i} = d_{i} \quad \forall i \in \{1,...,n\} \\
&\xi_{i j} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} \\
&y_{j} \in \{0,1\} \quad \forall j \in \{1,...,m\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j}$= número de itens tipo $i$ adicionados na mochila $j$


In [107]:
%%ampl_eval
reset;

In [108]:
%%ampl_eval
param n;
param cv;
param cp;
param np;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
param w {I} > 0;
param v {I} > 0;
param inner{I} > 0;
param d {I} > 0;
param maxVal := max {i in I} w[i];
param soma := sum{i in I} d[i];
param maxbins := ceil(soma / floor(cp / maxVal));

set J = 1..maxbins;

var e {I,J} integer >= 0;
var y {J} binary;

minimize FO:  sum {j in J} y[j];

s.t. R1_capacidade_mochila_peso {j in J}:
   sum {i in I} w[i] * e[i,j] * inner[i] <= cp * y[j] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J}:
   sum {i in I} v[i] * e[i,j] * inner[i] <= cv * y[j] suffix block j;

s.t. R3_capacidade_mochila_pecas {j in J}:
   sum {i in I} e[i,j] * inner[i] <= np * y[j] suffix block j;

s.t. R4_quantidade_demanda {i in I}:
   sum {j in J} e[i, j]* inner[i] = d[i] suffix block i;


### 1.4.1- Importar dados

In [109]:
df_process_situacao4 = df_process.query('chave_loja_buffer_onda == "2-CX-9"')
df_process_situacao4

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade,pai-cor,Volume_unit,Volume_total,Peso_total,chave_loja_buffer_onda
149,149,277,2023-11-14,768650318,300,172,89,0.260,24,CARTON_M,...,54.0,12,4,11,2.720000e+10,268-54.0,4592400,110217600,6.240,2-CX-9
221,221,407,2023-11-11,807049518,448,196,24,0.215,8,CARTON_M,...,18.0,6,2,11,2.720000e+10,391-18.0,2107392,16859136,1.720,2-CX-9
590,590,1027,2023-11-07,883874958,440,191,19,0.113,5,CARTON_M,...,124.0,5,1,11,2.720000e+10,433-124.0,1596760,7983800,0.565,2-CX-9
670,670,1172,2023-11-14,552909458,300,297,30,0.220,1,CARTON_M,...,3.0,3,1,11,2.720000e+10,151-3.0,2673000,2673000,0.220,2-CX-9
763,763,1172,2023-11-03,542140534,300,259,25,0.230,5,CARTON_M,...,74.0,1,1,11,2.720000e+10,135-74.0,1942500,9712500,1.150,2-CX-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29581,29581,6690,2023-11-27,739849299,314,198,110,0.840,2,CARTON_M,...,53.0,52,1,11,2.720000e+10,360-53.0,6838920,13677840,1.680,2-CX-9
29690,29690,7034,2023-11-13,597109315,330,201,128,0.700,2,CARTON_M,...,42.0,18,1,11,2.720000e+10,1359-42.0,8490240,16980480,1.400,2-CX-9
29901,29901,6967,2023-11-14,625821175,300,174,12,0.078,32,CARTON_M,...,18.0,17,4,11,2.720000e+10,2354-18.0,626400,20044800,2.496,2-CX-9
30685,30685,7169,2023-11-13,550943311,325,201,116,0.620,3,CARTON_M,...,306.0,21,1,11,2.720000e+10,865-306.0,7577700,22733100,1.860,2-CX-9


In [110]:
ids =  list(df_process_situacao4['Id'].values)
ids

[149,
 221,
 590,
 670,
 763,
 788,
 879,
 1391,
 1447,
 1572,
 1694,
 1884,
 2041,
 2302,
 2382,
 2466,
 2755,
 3729,
 3790,
 3793,
 3944,
 5396,
 5452,
 7142,
 7342,
 7540,
 7831,
 7896,
 7914,
 8244,
 8311,
 8650,
 8684,
 8704,
 8866,
 9060,
 9073,
 9083,
 9099,
 9144,
 9498,
 9656,
 9775,
 9900,
 10311,
 10312,
 10324,
 10409,
 10493,
 11093,
 11098,
 11419,
 11555,
 13723,
 14099,
 14609,
 14709,
 14846,
 15665,
 15686,
 15790,
 15879,
 16301,
 16315,
 16443,
 16777,
 16846,
 16915,
 16922,
 17237,
 18577,
 18950,
 19531,
 20568,
 21204,
 21367,
 21686,
 21874,
 21895,
 22215,
 22216,
 22380,
 22571,
 22618,
 22768,
 22818,
 22977,
 22990,
 23009,
 23381,
 24353,
 24475,
 24611,
 24898,
 24996,
 25120,
 26031,
 27238,
 27267,
 27502,
 27791,
 27900,
 27920,
 28244,
 29072,
 29077,
 29442,
 29581,
 29690,
 29901,
 30685,
 31430]

### 1.4.2- Entrada de dados no modelo

In [111]:
peso =  list(df_process_situacao4['Peso_unit'].values)
pecas = list(df_process_situacao4['Peças'].values)
volume = list(df_process_situacao4['Volume_unit'].values)
inner = list(df_process_situacao4['Inner'].values)

In [112]:
buffer = df_process_situacao4['Tipo_de_buffer'].unique()[0]
cp = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['Peso_max_(kg)'].values
cv = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['volume_util_caixa'].values
np = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['Peças_max'].values

print(cp, cv, np)

[23] [83393894.39999999] [170]


In [113]:
ampl.param["n"] = len(pecas)

ampl.param["cp"] = cp
ampl.param["cv"] = cv
ampl.param["np"] = np

w = peso
v = volume
d = pecas
inner = inner

ampl.param["w"] = {i: w[i - 1] for i in range(1, len(w) + 1)}
ampl.param["v"] = {i: v[i - 1] for i in range(1, len(v) + 1)}
ampl.param["inner"] = {i: inner[i - 1] for i in range(1, len(inner) + 1)}
ampl.param["d"] = {i: d[i - 1] for i in range(1, len(d) + 1)}

In [114]:
ampl.getValue('maxVal')

0.98

In [115]:
ampl.getValue('soma')

913

In [116]:
ampl.getValue('maxbins')

40

### Outros Solver

#### Solver GCG

In [117]:
ampl.option["solver"] = "gcg"
ampl.option["gcg_options"] = "outlev=1 timelimit=240"
ampl.solve()

GCG 4.0.0:   tech:outlev = 1
  lim:time = 240
 added complete decomp for original problem with 112 blocks and 0 masterconss, 4361 linkingvars, 0 mastervars, and max white score of   0.017394 
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 21 upgd conss, 0 impls, 21 clqs
   (0.0s) probing cycle finished: starting next cycle
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 1560 implications, 861 cliques
presolved problem has 4520 variables (880 bin, 3640 int, 0 impl, 0 cont) and 232 constraints
     21 constraints of type <setppc>
    211 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00
 calculated translation; number of missing constraints: 0; number of other partialdecs: 1 
Preexisting decomposition found. Solution proces

In [118]:
%%ampl_eval
display {j in J} y[j];
display {i in I, j in J} e[i,j];

y[j] [*] :=
 1 1    5 1    9 1   13 1   17 1   21 1   25 0   29 0   33 0   37 1
 2 1    6 1   10 1   14 1   18 1   22 0   26 0   30 0   34 0   38 1
 3 1    7 1   11 1   15 1   19 1   23 0   27 0   31 0   35 0   39 1
 4 1    8 1   12 1   16 1   20 1   24 0   28 0   32 0   36 0   40 1
;

e[i,j] [*,*]
:     1    2    3   4   5   6    7    8   9    10  11   12  13   14  15   16 :=
1      0    0   0   0   0    0    0   0    0    0   0    0   0    0   0    0
2      4    0   0   0   0    0    0   0    0    0   0    0   0    0   0    0
3      5    0   0   0   0    0    0   0    0    0   0    0   0    0   0    0
4      0    0   0   0   0    0    0   0    0    0   0    0   0    0   0    0
5      5    0   0   0   0    0    0   0    0    0   0    0   0    0   0    0
6      5    0   0   0   0    0    0   0    0    0   0    0   0    0   0    0
7     12    0   0   0   0    0    0   0    0    0   0    0   0    0   0    0
8      7    0   0   0   0    0    0   0    0    0   0    0   0    0   0    0
9   

In [120]:
e_var_s4_GCG = ampl.getVariable('e').getValues().toPandas()
e_var_s4_GCG = e_var_s4_GCG.reset_index()
e_var_s4_GCG.columns = ['itens','caixa', 'quantidade']

resultado_s4_GCG = e_var_s4_GCG.pivot(index='itens', columns='caixa', values='quantidade')


resultado_s4_GCG = resultado_s4_GCG.reset_index()
resultado_s4_GCG.columns.name = None
resultado_s4_GCG['id'] = ids
resultado_s4_GCG['pecas'] = pecas
resultado_s4_GCG['inner'] = inner

resultado_s4_GCG

,itens,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,id,pecas,inner
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,4,2,0,0,149,24,4
1,2,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,221,8,2
2,3,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,590,5,1
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,670,1,1
4,5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,763,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,108,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,29581,2,1
108,109,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,29690,2,1
109,110,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,29901,32,4
110,111,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,30685,3,1


In [121]:
resultado_final_s4_GCG = resultado_s4_GCG.loc[:, (resultado_s4_GCG != 0).any(axis=0)]
with pd.option_context('display.max_rows', None):
    display(resultado_final_s4_GCG)

,itens,1,2,3,4,5,6,7,8,9,...,19,20,21,37,38,39,40,id,pecas,inner
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,4,2,0,0,149,24,4
1,2,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,221,8,2
2,3,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,590,5,1
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,670,1,1
4,5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,763,5,1
5,6,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,788,10,2
6,7,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,879,12,1
7,8,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1391,21,3
8,9,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1447,2,1
9,10,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1572,3,1


In [122]:
resultado_final_s4_GCG['soma_multiplo'] = resultado_final_s4_GCG['pecas']/resultado_final_s4_GCG['inner']
resultado_final_s4_GCG['soma_multiplo']

0       6.0
1       4.0
2       5.0
3       1.0
4       5.0
       ... 
107     2.0
108     2.0
109     8.0
110     3.0
111    16.0
Name: soma_multiplo, Length: 112, dtype: float64

In [123]:
for i in resultado_final_s4_GCG.drop(columns=['itens', 'id', 'inner', 'pecas']).columns.values:
    print(f"soma de múltiplo {i}: {resultado_final_s4_GCG[i].sum()}")

soma de múltiplo 1: 46
soma de múltiplo 2: 28
soma de múltiplo 3: 27
soma de múltiplo 4: 17
soma de múltiplo 5: 11
soma de múltiplo 6: 32
soma de múltiplo 7: 36
soma de múltiplo 8: 21
soma de múltiplo 9: 35
soma de múltiplo 10: 33
soma de múltiplo 11: 34
soma de múltiplo 12: 15
soma de múltiplo 13: 24
soma de múltiplo 14: 40
soma de múltiplo 15: 20
soma de múltiplo 16: 34
soma de múltiplo 17: 16
soma de múltiplo 18: 34
soma de múltiplo 19: 9
soma de múltiplo 20: 13
soma de múltiplo 21: 16
soma de múltiplo 37: 4
soma de múltiplo 38: 2
soma de múltiplo 39: 10
soma de múltiplo 40: 11
soma de múltiplo soma_multiplo: 568.0


In [125]:
total_soma_resultado = 0

for i in resultado_final_s4_GCG.drop(columns=['itens', 'id', 'inner', 'pecas', 'soma_multiplo']).columns.values:
    total_soma_resultado += resultado_final_s4_GCG[i].sum()

print(total_soma_resultado)

568


In [126]:
print("número de caixas:",len(resultado_final_s4_GCG.drop(columns=['itens', 'id', 'inner', 'pecas', 'soma_multiplo']).columns))

número de caixas: 25
